In [1]:
import os
import re
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(69)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
with open('data/kon.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))

length of dataset in characters:  493412


In [3]:
print(text[:500])

Ui:
Sis, come on. You'd better get out of bed. Sis?

Yui:
Ah! I-it's eight! I'm late! Oh!

Ui:
Hey, why the rush? Hm?

Yui:
See you later!

Lady:
Oh, good morning, Yui.

Yui:
Good morning!

Yui:
What?! I read the clock wrong!
Starting today, I'm a high schooler!

Opening Song
Cagayake!GIRLS by 放課後ティータイム(After School Tea Time)

Girls:
Congratulations on starting school here!

Girl 1:
Please join the Tennis Club!

Girl 2:
The Judo Club's better!

Girl 3:
Please join the Tea Ceremony Club!

Girl 4:


In [4]:
# remove japanese characters
text = ''.join(filter(lambda character:ord(character) < 0x3000, text))

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("unique characters:", vocab_size, ''.join(chars))

unique characters: 93 
 !"#$%&'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz{|}~°éū‘’…♪


In [6]:
# Very simple tokenizer
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
# add special token for padding
stoi[''] = len(stoi)
itos[len(itos)] = ''
print(stoi)
print(itos)
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])
print("encoded:", encode(text[:20]))
print("decoded:", decode(encode(text[:20])))
vocab_size = len(itos)
print("vocab size:", vocab_size)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, 'A': 28, 'B': 29, 'C': 30, 'D': 31, 'E': 32, 'F': 33, 'G': 34, 'H': 35, 'I': 36, 'J': 37, 'K': 38, 'L': 39, 'M': 40, 'N': 41, 'O': 42, 'P': 43, 'Q': 44, 'R': 45, 'S': 46, 'T': 47, 'U': 48, 'V': 49, 'W': 50, 'X': 51, 'Y': 52, 'Z': 53, '[': 54, ']': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81, '{': 82, '|': 83, '}': 84, '~': 85, '°': 86, 'é': 87, 'ū': 88, '‘': 89, '’': 90, '…': 91, '♪': 92, '': 93}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '$', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 

In [7]:
data = torch.tensor(encode(text), dtype=torch.int64)
data.to(device)
data.shape

torch.Size([493171])

In [8]:
data[:100]

tensor([48, 64, 25,  0, 46, 64, 74, 11,  1, 58, 70, 68, 60,  1, 70, 69, 13,  1,
        52, 70, 76,  8, 59,  1, 57, 60, 75, 75, 60, 73,  1, 62, 60, 75,  1, 70,
        76, 75,  1, 70, 61,  1, 57, 60, 59, 13,  1, 46, 64, 74, 27,  0,  0, 52,
        76, 64, 25,  0, 28, 63,  2,  1, 36, 12, 64, 75,  8, 74,  1, 60, 64, 62,
        63, 75,  2,  1, 36,  8, 68,  1, 67, 56, 75, 60,  2,  1, 42, 63,  2,  0,
         0, 48, 64, 25,  0, 35, 60, 80, 11,  1])

In [9]:
n = int(len(data) * 0.95)
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([468512]) torch.Size([24659])


In [10]:
block_size = 8
train_data[:block_size+1]

tensor([48, 64, 25,  0, 46, 64, 74, 11,  1])

In [11]:
# context and target simulation
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1].tolist()
    target = y[t].item()
    print('context:', context, 'target:', target)

context: [48] target: 64
context: [48, 64] target: 25
context: [48, 64, 25] target: 0
context: [48, 64, 25, 0] target: 46
context: [48, 64, 25, 0, 46] target: 64
context: [48, 64, 25, 0, 46, 64] target: 74
context: [48, 64, 25, 0, 46, 64, 74] target: 11
context: [48, 64, 25, 0, 46, 64, 74, 11] target: 1


In [12]:
torch.manual_seed(69)
batch_size = 4 # number of parallel blocks
block_size = 8 # number of characters in each block = context length

def get_batch(split, block_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train', 128)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 128])
tensor([[66, 70, 25,  0, 40, 64, 74, 74,  1, 52, 56, 68, 56, 69, 56, 66, 56,  1,
         64, 74,  1, 73, 60, 56, 67, 67, 80,  1, 62, 73, 60, 56, 75,  2,  0,  0,
         41, 70, 57, 76, 80, 70, 25,  0, 36, 75,  8, 74,  1, 67, 64, 66, 60,  1,
         74, 63, 60,  8, 74,  1, 62, 60, 75, 75, 64, 69, 62,  1, 71, 73, 60, 75,
         75, 64, 60, 73,  1, 56, 69, 59,  1, 71, 73, 60, 75, 75, 64, 60, 73,  1,
         60, 77, 60, 73, 80,  1, 59, 56, 80,  2,  0,  0, 46, 56, 78, 56, 66, 70,
         25,  0, 54, 62, 64, 62, 62, 67, 60, 74, 55,  0,  0, 40, 64, 70, 25,  0,
          9, 36],
        [70, 75,  0,  0, 40, 64, 70, 25,  0, 39, 60, 75,  8, 74,  1, 74, 60, 60,
         13, 13, 13,  0,  0, 45, 64, 75, 74, 76, 25,  0, 48, 63, 11,  1, 64, 75,
          8, 74,  1, 69, 70, 75, 63, 64, 69, 62,  2,  0, 30, 63, 60, 58, 66,  1,
         64, 75,  2,  0, 40, 70, 76, 74, 75, 56, 58, 63, 60,  2,  0,  0, 47, 74,
         76, 68, 76, 62, 64, 25,  0, 36,  1, 63, 56, 77, 60,  

In [13]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # Bigram language model: single layer, single token prediction
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C), B: batch=4, T: sequence=8, C: vocab=147

        if targets is None:
            loss = None
        else:
            # flatten the logits and targets for torch cross entropy
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # generate max_new_tokens new tokens given the initial context idx
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# randomly generate 100 tokens from initial model weights and idx = 0 = \n
print(decode(m.generate(idx=torch.zeros(
    (1, 1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

torch.Size([512, 94])
tensor(4.8468, grad_fn=<NllLossBackward0>)

wshZL} {°vr9Wx]X21?V(;b2?|°Xldiéé1gvEk(SI.mM1:'uk??{aEmfx8UE :…P,V}UV8rxéOK% 1!nEEpH;|qWv04;♪' nisF|


In [14]:
# training!
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(5000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4681079387664795


In [15]:
# generate 100 tokens from trained model weights and idx = 0 = \n
print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=device),
                   max_new_tokens=300)[0].tolist()))



Yu?

I t iJo hollooou jeUindu ps t?
NOhsoga tsankemeatherd y I "som?! lw...


TheAtZwe wn supou:'oneat am[4, acive ack?
I l?{}Rey st'r e,Ju'tt jui:

I'roCa?
Y…PMi:

Yusu.

Y…I okn-muigheab f lKatugnngif ws mowhalquitsI'4;u:


Yumscton'sas yowd Misughe62VWhamioinno i:
BESa'YHuthy 5]Qaneng orllealéx t


Lets try out lower dimensional embeddings + positional embeddings

In [16]:
class BigramEmbedLanguageModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size):
        super().__init__()
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from 
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T,C) [0...T-1]
        x = tok_embd + pos_embd
        logits = self.lm_head(x) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [17]:
# training!
model = BigramEmbedLanguageModel(vocab_size, 16, 32)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(5000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3653781414031982


In [18]:
# generate
print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=device),
                   max_new_tokens=300)[0].tolist()))



Yuithane the ghhuthe ayitiste yid g um.. che l n'tonk lopar Pallkaugory fan.
Thapriou'yatha thingourimmere y-ceve anot sumyoror Thy!?
Shodonanony ly, oushako as s meveve oonave iti:
Mingo w bd t'me i:

Ohele t hecasiveve've's n illd ithalve hat't.
Al.
Mme dtui:
Te whingetse.
O{? Th g atomotind derse


# Now for the Transformer Fundamentals!

## The mathematical trick to self-attention: triangular matrices for weighted averages

In [19]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 2,8,2 # batch, time, channels
x = torch.randn(B,T,C)
print(x.shape)
print(x)

torch.Size([2, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]]])


In [20]:
# We want x[b,t] = mean_{i<=t} x[b,i] to very badly encode info of tokens before token t
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]])

In [21]:
# better way to do this: triangular matrix!
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print(wei)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
xbow2

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]])

In [22]:
# even better: softmax for normalization of weights
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
xbow3

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]])

In [23]:
# finally: Query (what i look for), Key (What i am in this), 
# Value (My private value, embedded) for self-attention
# version 4: single head self-attention
head_size = 16
query = nn.Linear(C, head_size, bias=False) # Linear layer C (embed) -> head size (16)
key = nn.Linear(C, head_size, bias=False) # Linear layer C (embed) -> head size (16)
value = nn.Linear(C, head_size, bias=False) # Linear layer C (embed) -> head size (16)

q = query(x) # (B,T,16)
k = key(x) # (B,T,16)
wei = q @ k.transpose(-2, -1) # (B,T,16) @ (B,16,T) -> (B,T,T)
# print(wei.shape)
# print(torch.round(torch.sum(wei, dim=1), decimals=3))
# row_sum = wei.sum(dim=1)

# # Compute the average sum
# avg_sum = row_sum.mean()

# # Filter out rows with sum lower than the average sum
# wei = wei[:, row_sum >= avg_sum]
# print(wei.shape)

wei = wei * C**-0.5 # scaled attention as to not sharpen softmax

# T = wei.shape[1]
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(torch.round(wei, decimals=3))

v = value(x)
out = wei @ v
print(torch.round(out, decimals=3))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4700, 0.5300, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3140, 0.3170, 0.3690, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2060, 0.2090, 0.2640, 0.3210, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1920, 0.1650, 0.2050, 0.2140, 0.2250, 0.0000, 0.0000, 0.0000],
         [0.1260, 0.1320, 0.0900, 0.0690, 0.2020, 0.3810, 0.0000, 0.0000],
         [0.1440, 0.1500, 0.1540, 0.1630, 0.1220, 0.1160, 0.1500, 0.0000],
         [0.0580, 0.0460, 0.0440, 0.0340, 0.1330, 0.1390, 0.0480, 0.4990]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4970, 0.5030, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0040, 0.0540, 0.9430, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5030, 0.1000, 0.0140, 0.3840, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2570, 0.1220, 0.0820, 0.1950, 0.3440, 0.0000, 0.0000, 0.0000],
         [0.0330, 0.121

# Time to put attention in our last model!

In [24]:
class SelfAttentionHead(nn.Module):
    """ one head of self-attention """
    def __init__(self, block_size, n_embd, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size, device=device)))

    def forward(self, x):
        B,T,C = x.shape
        q = self.query(x) # (B,T,C)
        k = self.key(x)   # (B,T,C)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei * C**-0.5 # scaled attention
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)

        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [25]:
class BigramEmbedAttentionLanguageModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size):
        super().__init__()
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # single head self-attention
        self.sa_head = SelfAttentionHead(block_size, embed_size, embed_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd
        # apply self-attention
        x = self.sa_head(x)
        # get logits with linear layer
        logits = self.lm_head(x) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [26]:
# training!
model = BigramEmbedAttentionLanguageModel(vocab_size, 16, 32)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(5000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())

learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
2.3798601627349854


In [27]:
print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=device),
                   max_new_tokens=500)[0].tolist()))




Tuh:
Thouiges se tomy?

Mii:
Ir! ea...

O:
Aa..

Noon.!

Azusng
Yu.

Ritseat.
Hmu sha:
Fu?

Se igo's She ot a?

Yui:

Ritt't nawis hene, bng oo:
I'ctm thea kn, ame intsetis sogigah:
Azsn's the unbe..

Miits or hwe.... 4oviet!
O ups hexlad heys a lpe ly!

Fry feim!

Mio:
Whe, baly ont idn glapl tther umim!

Yui:

Yut hayu ish! yhsi:
Yen dereree cheve hse gughin't srea:
Oh! wak!

Ritngqoiger.. Sa Heh.

You'p fing ouks stitas okeare. ten cerenn to? Azourmuth!

Ritshengy Ts Ahawed of xe wis!

Tumuh


# More heads! Multi-Head Attention

In [28]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, block_size, num_heads, n_embd, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(block_size, n_embd, head_size) for _ in range(num_heads)])

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concat single-head results
        return out

In [29]:
class BigramEmbedMultiHeadAttentionLanguageModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size, head_num):
        super().__init__()
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # multi-head self-attention
        self.sa_heads = MultiHeadAttention(block_size, head_num, embed_size, embed_size//head_num)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd
        # apply multi-head self-attention
        x = self.sa_heads(x)
        # get logits with linear layer
        logits = self.lm_head(x) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [30]:
# training!
model = BigramEmbedMultiHeadAttentionLanguageModel(vocab_size, 16, 32, 4)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())

learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
learning step: 5000
learning step: 5500
learning step: 6000
learning step: 6500
learning step: 7000
learning step: 7500
learning step: 8000
learning step: 8500
learning step: 9000
learning step: 9500
1.841754674911499


In [31]:
idx = encode("Azusa:\n")
print(torch.tensor([idx]))
print(
    decode(
        m.generate(idx=torch.tensor([idx], dtype=torch.long, device=device),
                   max_new_tokens=500)[0].tolist()))


tensor([[28, 81, 76, 74, 56, 25,  0]])
Azusa:
You lay!

Mio:
I'm ay trnakan nee gon herembiooond I monn't ferelly sthoul! Mis]

Ritsums octon rlind shes]

Mio:
Shat!

Azuce it dind alld the? So to heplins you nthis azpainiden!

Mio:
Allmy "ywan.

Ritsu:
Oh, nerry uith! Welling Geply thels guing]

Azusa:
Oh?

Meve Saw the ppas! Weadk.

Azusa:
Huh? I'm then?

Mio:
Heloo thictickanne hoon (d hover!

Mio:
Clugid you Glipre?

Mio:
Pllly Uiprgod the in next a stidery you ju'll, that sory get?

Yui:
Ohs bet a forfrougi-I onnk tonte fustsu:
I the's


# Time to think: Feed-Forward to compute attention results

In [32]:
class FeedForward(nn.Module):
    def __init__(self, n_embed, n_hidden):
        super().__init__()
        self.lin_1 = nn.Linear(n_embed, n_hidden)
        self.lin_2 = nn.Linear(n_hidden, n_embed)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.lin_1(x)
        x = self.relu(x)
        x = self.lin_2(x)
        return x

In [33]:
class BigramEmbedMultiHeadAttentionFeedForwardLanguageModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size, head_num):
        super().__init__()
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # multi-head self-attention
        self.sa_heads = MultiHeadAttention(block_size, head_num, embed_size, embed_size//head_num)
        # feed forward
        self.ff_layer = FeedForward(embed_size, 128)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd
        # apply multi-head self-attention
        x = self.sa_heads(x)
        # feed forward
        x = self.ff_layer(x)
        # get logits with linear layer
        logits = self.lm_head(x) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [34]:
# training!
model = BigramEmbedMultiHeadAttentionFeedForwardLanguageModel(vocab_size, 16, 32, 4)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())

learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
learning step: 5000
learning step: 5500
learning step: 6000
learning step: 6500
learning step: 7000
learning step: 7500
learning step: 8000
learning step: 8500
learning step: 9000
learning step: 9500
1.660374641418457


In [35]:
idx = encode("Azusa:\n")
print(torch.tensor([idx]))
print(
    decode(
        m.generate(idx=torch.tensor([idx], dtype=torch.long, device=device),
                   max_new_tokens=500)[0].tolist()))


tensor([[28, 81, 76, 74, 56, 25,  0]])
Azusa:
Afl do to has Cirmy 2aLy?! (Hey, you had truster:
Anyay thish, Muse idaye to laparty big tary.

You colk is of heres, Azusa:
If Huh, Yui's quitsusa:
Alted.

Yui:
Shanks uter:
How.

Ritsu:
Ses like a so cime? AWh, and much, I do you the aft our!

Yui& Ritsu:
Hey, chaps.

Mio:
All? Owh!

Mio:
Oh. What, work, I'll, Sams to he forkry, right doe's sistudy don't too this his to the to it. You chat day'm azich come forry turgotury.. 
Ples as right wow.

Mio:
Oh! Hery at finirse, to by tit is to ready e


In [36]:
total_params = sum(p.numel() for p in m.parameters() if p.requires_grad)
total_params

18046

# Make it scalable: repeatable Blocks

In [37]:
class Block(nn.Module):
    def __init__(self, block_size, n_heads, n_embd):
        super().__init__()
        self.sa_heads = MultiHeadAttention(block_size, n_heads, n_embd, n_embd//n_heads)
        self.ff_layer = FeedForward(n_embd, 128)
    
    def forward(self, x):
        x = self.sa_heads(x)
        x = self.ff_layer(x)
        return x

In [38]:
class TransformerNoResidualNoNormModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size, head_num, layer_num):
        super().__init__()
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # transformer blocks
        self.blocks = nn.Sequential(*[Block(block_size, head_num, embed_size) for _ in range(layer_num)])

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd
        # go through blocks
        x = self.blocks(x)
        # get logits with linear layer
        logits = self.lm_head(x) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [55]:
device

'cpu'

In [57]:
# training!
model = TransformerNoResidualNoNormModel(vocab_size, 32, 64, 8, 4)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', block_size=32)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())

learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
learning step: 5000
learning step: 5500
learning step: 6000
learning step: 6500
learning step: 7000
learning step: 7500
learning step: 8000
learning step: 8500
learning step: 9000
learning step: 9500
2.1401207447052


In [58]:
print(loss.item())


2.1401207447052


In [59]:
idx = encode("Yui:\n")
print(torch.tensor([idx]))
print(
    decode(
        m.generate(idx=torch.tensor([idx], dtype=torch.long, device=device),
                   max_new_tokens=1000)[0].tolist()))


tensor([[52, 76, 64, 25,  0]])
Yui:
Aw srua! Wc den aler Yo a fe. I amy'r fule wis nughts

Ui:
[h was Wturise

Uih & oadi
Ada:
Th caz Ie rav ly we tonep, lash here ni.

Rzumugigi:
We dik!

Ritsu:
[h.
Msit Os. Weut i lamett myet ex fet nand you'r liin wardyt lalle irealt ne't t then, you illy fastssit at qeating I'd'le wanr (n wea srat, ofo  thisk eat ape mass cetay srape lkann.

Mumtu:
Hef't innd weany? W dhan uz ish ropf be'm tha-y,itingy3!

Yui:
"Whokat dtont Iam a cad iken, alor Jdeilipe ellllind n to goys.

So:
Arh. hamt cle saate "ote sook sitk fake tealt nout afe u drind.

Yui:
Houlldg

Yui:
Oh! Morh,. Kho?

Azusa:
Tac]. Iane wou herely bite thoore uf rip?

Tsumuging, airling?

Sodoko:
Oh,s..

Tsumugi:
Uah,sp I soa mono't dani wouree citereng geio-gM ofeb mou!

Juno:
Fe hit we okhirell tid g. Lig.

Azusa:
Azat moulec?

Yui:
Pow you mo mam gorena yersy ouw mo suy ou high Raye an to b&
Sooda:
Ar-n sgAed fersy.

Tsumugi:
Te eeod

Ritsu:
Whing ro?

Ritsu:
Thut yan sei.

Pan

Tsumugig

In [42]:
total_params = sum(p.numel() for p in m.parameters() if p.requires_grad)
total_params

23390

# Trying out fading blocks

In [43]:
class Fade(nn.Module):
    def __init__(self, n_input):
        super().__init__()
        n_output = n_input//2
        self.out_sizes = [n_output//8, n_output//8, n_output//4, n_output//2]
        n_rest = n_input - n_output//2
        self.in_sizes = [n_rest//2, n_rest//4, n_rest//4, n_output//2]
        self.lin1 = nn.Linear(self.in_sizes[0], self.out_sizes[0], bias=False)
        self.lin2 = nn.Linear(self.in_sizes[1], self.out_sizes[1], bias=False)
        self.lin3 = nn.Linear(self.in_sizes[2], self.out_sizes[2], bias=False)

    def forward(self, x): # (B, T, C)
        # turn x to (B, C, T)
        x4 = x[:, -self.in_sizes[3]:]
        x = x.transpose(1, 2)
        x1 = self.lin1(x[:, :, :self.in_sizes[0]])
        x2 = self.lin2(x[:, :, self.in_sizes[0]:self.in_sizes[0]+self.in_sizes[1]])
        x3 = self.lin3(x[:, :, self.in_sizes[0]+self.in_sizes[1]:self.in_sizes[0]+self.in_sizes[1]+self.in_sizes[2]])
        # turn back to (B, T/2, C)
        x1 = x1.transpose(1, 2)
        x2 = x2.transpose(1, 2)
        x3 = x3.transpose(1, 2)
        x = torch.cat((x1, x2, x3, x4), dim=1)
        return x

# test fade
x = torch.randn(2, 32, 3)
print(x.shape)
print(x[0, :, 0])
f = Fade(32)
y = f(x)
print(y.shape)
print(y)


torch.Size([2, 32, 3])
tensor([ 1.7517, -0.2906, -2.0363,  0.0255,  0.7507, -1.6608, -0.4314,  0.0462,
         0.9031, -1.4160,  0.3234, -0.3443,  0.0070, -1.7629, -1.2455,  0.7949,
        -0.4685,  0.2199, -1.2016,  0.7289, -0.4999,  1.0719,  0.5003, -0.3971,
         0.7200,  1.7006,  1.7204,  0.6477, -2.3513,  1.4432,  2.3435,  0.2770])
torch.Size([2, 16, 3])
tensor([[[ 0.0076, -0.7503,  0.2154],
         [-1.0325, -0.0345, -0.4802],
         [ 0.4516,  0.1592, -0.4316],
         [ 0.1925, -0.2216, -0.1444],
         [ 0.0732,  0.0355,  0.0430],
         [ 0.0498, -0.4532,  0.3876],
         [-0.4966,  0.5192, -0.3662],
         [-0.4446, -0.4080, -0.0272],
         [ 0.7200, -0.8576,  0.5261],
         [ 1.7006,  1.4979,  0.1056],
         [ 1.7204, -0.4747,  1.5703],
         [ 0.6477, -0.9721, -1.1710],
         [-2.3513, -0.9021, -0.0137],
         [ 1.4432,  0.0752, -1.4775],
         [ 2.3435, -0.2121, -2.0091],
         [ 0.2770, -0.5056, -0.0235]],

        [[ 0.3494, -0.0

In [44]:
y


tensor([[[ 0.0076, -0.7503,  0.2154],
         [-1.0325, -0.0345, -0.4802],
         [ 0.4516,  0.1592, -0.4316],
         [ 0.1925, -0.2216, -0.1444],
         [ 0.0732,  0.0355,  0.0430],
         [ 0.0498, -0.4532,  0.3876],
         [-0.4966,  0.5192, -0.3662],
         [-0.4446, -0.4080, -0.0272],
         [ 0.7200, -0.8576,  0.5261],
         [ 1.7006,  1.4979,  0.1056],
         [ 1.7204, -0.4747,  1.5703],
         [ 0.6477, -0.9721, -1.1710],
         [-2.3513, -0.9021, -0.0137],
         [ 1.4432,  0.0752, -1.4775],
         [ 2.3435, -0.2121, -2.0091],
         [ 0.2770, -0.5056, -0.0235]],

        [[ 0.3494, -0.0192, -0.5864],
         [-0.1919, -0.3533, -0.1036],
         [-0.6943, -0.3697,  0.1478],
         [ 0.6035, -0.7517, -0.2908],
         [ 0.2768,  0.3092,  0.1300],
         [ 0.9684, -0.6234,  0.2205],
         [-0.3864,  0.0371, -0.0960],
         [-0.0281,  0.2589, -0.4893],
         [-1.2999,  0.2286, -0.7944],
         [-1.0271, -0.7806, -0.8205],
         [

In [45]:
def calc_fade(n_input):
    fade_steps = [n_input]
    while n_input > 16:
        n_output = n_input//2
        fade_steps.append(n_output)
        n_input = n_output
    return fade_steps

calc_fade(512)

[512, 256, 128, 64, 32, 16]

In [46]:
class FadingBlock(nn.Module):
    def __init__(self, block_size, n_heads, n_embd, n_time):
        super().__init__()
        self.sa_heads = MultiHeadAttention(block_size, n_heads, n_embd, n_embd//n_heads)
        self.ff_layer = FeedForward(n_embd, 128)
        self.fade = Fade(n_time)

    def forward(self, x):
        x = self.sa_heads(x)
        x = self.ff_layer(x)
        x = self.fade(x)
        return x


In [47]:
def pad_encoded(x, block_size, vocab_size):
    # add zeros before x to make it block_size, x is list of ints
    return [vocab_size-1]*(block_size-len(x)) + x

In [48]:
class FadeFormerNoResidualNoNormModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size, head_num):
        super().__init__()
        self.block_size = block_size
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(self.block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # calculate fade n_time
        fade_ins = calc_fade(self.block_size)
        # transformer blocks
        self.blocks = nn.Sequential(
            *[FadingBlock(block_size, head_num, embed_size, fade_in) for fade_in in fade_ins])

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx)  # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd # (B,T,C) + (T,C) -> (B,T,C)
        # go through blocks
        x = self.blocks(x)
        # get logits with linear layer
        logits = self.lm_head(x)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets[:, -8:]
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

model = FadeFormerNoResidualNoNormModel(vocab_size, 128, 64, 8)
m = model.to(device)
idx = encode("Azusa:\n")
padded_idx = pad_encoded(idx, 128, vocab_size)
print(decode(padded_idx))
print(
    decode(
        m.generate(idx=torch.tensor([padded_idx],
                                    dtype=torch.long,
                                    device=device),
                   max_new_tokens=500)[0].tolist()))
print("model size:", sum(p.numel() for p in m.parameters()))


Azusa:

Azusa:
b♪qI4##yIrZCRL°2~A/'V
7vm"#b55S }/~~A?/-|GS'oLEs6c7K'ūjL(’94"JY(e PkcEhHd K:JCTYn9-Pd7&)YGo6m♪PSjjgbur& #8♪‘T'cy#ktGX}3aO7A}G1s5QHZC#Ffcé.D’oE}W(H?:0c)‘5M(/6ON-!uG{qa[9JT72QlOoJ5m pXxOPuf1Aez…NDeXF…QMq0xSS?Ab‘é3H"!hNP16oZUw/}TAM&S~"aZ}nd9sS~VF;[!(Q(/L/'q)9BéPPix…|&3e:59rl2TcDUnrmTEv}L/cHk6P-v?C°|’
e…Exb!"f,rMūk%W{]i°8fyvz(}(7"U0'xR0Rb0t…VyyjH&'KSBHajjūzWN wo6.H5%u#-) 2)B8vpxmHūm%’szn62J]h:LFé(hDH5aN2jC♪jo,%FRf~'hsq~ijO"vM?6m?jZ6f?}PwiW3’Q!‘1QMWGZ.AJ.e'éVsysG[Aū?'&#m48-Ww4knGū:lw!$v{)AtO2%&mw;
model size: 137049


In [49]:
# training!
model = FadeFormerNoResidualNoNormModel(vocab_size, 16, 16, 4)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())


learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
learning step: 5000
learning step: 5500
learning step: 6000
learning step: 6500
learning step: 7000
learning step: 7500
learning step: 8000
learning step: 8500
learning step: 9000
learning step: 9500
2.049769401550293


In [50]:
print(loss)
print("model size:", sum(p.numel() for p in m.parameters()))


tensor(2.0498, grad_fn=<NllLossBackward0>)
model size: 8381


In [51]:
idx = encode("Yui:\n")
print(torch.tensor([idx]))
padded_idx = pad_encoded(idx, 16, vocab_size)
print(
    decode(
        m.generate(idx=torch.tensor([padded_idx],
                                    dtype=torch.long,
                                    device=device),
                   max_new_tokens=1000)[0].tolist()))


tensor([[52, 76, 64, 25,  0]])
Yui:
I &e pd mase my Ur.

Ritsu:
Yo, Yui:
Yey No nligidst ou ap!

Ritsu:
Uh, alamed!

Mio:
It chuge?

Yui:
It Rus, oan bustl Mls ohargenke! bus?

It." Ritsugi:
I pod he sha fe is ythow] I thase sik.

Ritsu:
Ritdsuma:
Inpve tar sew planse. It cen anet orthe ang arly Chellot ord it.

Aoyom, Ruitse Jude is Yee that Musy yot fad llnd hant feoat'gr tusic senen Yay youpt?

Itou:
It Whanenmp, thet'nat he pritf hou le,..

Rrusu:
Puou Yeat inp shat I lboung thar yer thac herd.

Yui:
Hean keay a walk to fa Ildind gus. anding atines!

[ Whlot me therere somt ry chat anf sasher thd luis f Musicar tat!

Itsi!

Apsy. Gly, iy It mik chous.

Yui:
Idrki:
Hi bll teret ay yougeny s hak shat!

Yui:
Oh, Sawsas. hakp coen what'rchat thece sir ahe nerly. Ei o gon The ene ot!

Ritsu:
No semener wanrd?

Apa4!
High:
Where's ree-rasafe!

Yui:
Tsrehe… 'ave whe hang yow't asardeseakig these imciksins Fuy fagu cmils Henet yeanching ais a tonws gis.

Huftc, the outhe puisg uhr hat.

Yu

# Fade with residuals that concat at the end, perhaps?

In [52]:
class FadeWithResidual(nn.Module):
    def __init__(self, n_input):
        super().__init__()
        n_output = n_input//2
        self.out_sizes = [n_output//8, n_output//8, n_output//4, n_output//2]
        n_rest = n_input - n_output//2
        self.in_sizes = [n_rest//2, n_rest//4, n_rest//4, n_output//2]
        self.lin1 = nn.Linear(self.in_sizes[0], self.out_sizes[0], bias=False)
        self.lin2 = nn.Linear(self.in_sizes[1], self.out_sizes[1], bias=False)
        self.lin3 = nn.Linear(self.in_sizes[2], self.out_sizes[2], bias=False)

    def forward(self, x):  # (B, T, C)
        # turn x to (B, C, T)
        res = x[:, :x.shape[1]//2]
        x4 = x[:, -self.in_sizes[3]:]
        x = x.transpose(1, 2)
        x1 = self.lin1(x[:, :, :self.in_sizes[0]])
        x2 = self.lin2(x[:, :, self.in_sizes[0]:self.in_sizes[0]+self.in_sizes[1]])
        x3 = self.lin3(x[:, :, self.in_sizes[0]+self.in_sizes[1]:self.in_sizes[0]+self.in_sizes[1]+self.in_sizes[2]])
        # turn back to (B, T/2, C)
        x1 = x1.transpose(1, 2)
        x2 = x2.transpose(1, 2)
        x3 = x3.transpose(1, 2)
        x = torch.cat((x1, x2, x3, x4), dim=1)
        return res, x


# test fade
x = torch.randn(2, 16, 3)
# print(x.shape)
# print(x[0, :, 0])
f = FadeWithResidual(16)
res, y = f(x)
print(res, y)

tensor([[[ 1.2255, -0.5773,  1.0730],
         [ 1.5105, -1.2880, -0.7156],
         [ 1.4106, -2.2994,  0.1899],
         [-1.1967, -0.4112,  0.8287],
         [ 1.0561, -0.8631,  1.7145],
         [-1.8309, -0.7984, -0.9468],
         [ 1.3745, -1.0130,  2.0272],
         [-0.6549,  0.6922,  0.9853]],

        [[-0.5386,  0.7189,  0.6516],
         [-0.0509, -0.2531, -0.3140],
         [ 1.2580, -1.5122, -0.8516],
         [ 0.4114,  0.3068,  0.4953],
         [ 2.8530, -1.5954, -0.5194],
         [-0.5329,  0.8227,  0.6288],
         [-0.3114, -1.2955, -1.7756],
         [-0.0494, -0.6385,  0.9877]]]) tensor([[[ 0.4403, -0.5590, -0.5726],
         [ 0.8587, -0.1138,  1.5311],
         [-0.5020, -0.4448, -0.2279],
         [ 0.2643,  0.0642,  0.1293],
         [-0.5139,  0.6565, -0.8653],
         [-0.5227, -1.5890, -1.4470],
         [ 0.6933, -1.3955,  0.7972],
         [-0.8732, -1.1848, -0.1624]],

        [[-0.0992, -0.5138, -0.5226],
         [-0.0849, -0.6303, -0.4317],
      

In [53]:
class ResidualFadingBlock(nn.Module):
    def __init__(self, block_size, n_heads, n_embd, n_time):
        super().__init__()
        self.sa_heads = MultiHeadAttention(
            block_size, n_heads, n_embd, n_embd//n_heads)
        self.ff_layer = FeedForward(n_embd, 128)
        self.fade = FadeWithResidual(n_time)

    def forward(self, x):
        x = self.sa_heads(x)
        x = self.ff_layer(x)
        res, x = self.fade(x)
        return res, x


In [54]:
class ResidualFadeFormerNoNormModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size, head_num):
        super().__init__()
        self.block_size = block_size
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(
            self.block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # calculate fade n_time
        fade_ins = calc_fade(self.block_size)
        # transformer blocks
        self.blocks = nn.ModuleList()
        self.blocks.append(*[
            ResidualFadingBlock(block_size, head_num, embed_size, fade_in)
            for fade_in in fade_ins
        ])

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx)  # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd  # (B,T,C) + (T,C) -> (B,T,C)
        # go through blocks
        final = torch.tensor([], device=device)
        for block in self.blocks:
            res, x = block(x)
            final = torch.cat((final, res), dim=1)
        x = torch.cat((final, x), dim=1)
        # get logits with linear layer
        logits = self.lm_head(x)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = ResidualFadeFormerNoNormModel(vocab_size, 128, 64, 8)
m = model.to(device)
idx = encode("Azusa:\n")
padded_idx = pad_encoded(idx, 128, vocab_size)
print(decode(padded_idx))
print(
    decode(
        m.generate(idx=torch.tensor([padded_idx],
                                    dtype=torch.long,
                                    device=device),
                   max_new_tokens=500)[0].tolist()))
print("model size:", sum(p.numel() for p in m.parameters()))


TypeError: ModuleList.append() takes 2 positional arguments but 5 were given

In [ ]:
# training!
model = ResidualFadeFormerNoNormModel(vocab_size, 16, 16, 4)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())


learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
learning step: 5000
learning step: 5500
learning step: 6000
learning step: 6500
learning step: 7000
learning step: 7500
learning step: 8000
learning step: 8500
learning step: 9000
learning step: 9500
2.7385239601135254


In [ ]:
print(loss.grad_fn)
print("model size:", sum(p.numel() for p in m.parameters()))


model size: 3358


In [ ]:
for param in m.parameters():
    print(param.shape)


torch.Size([94, 16])
torch.Size([16, 16])
torch.Size([94, 16])
torch.Size([94])


In [ ]:
idx = encode("Yui:\n")
print(torch.tensor([idx]))
padded_idx = pad_encoded(idx, 16, vocab_size)
print(
    decode(
        m.generate(idx=torch.tensor([padded_idx],
                                    dtype=torch.long,
                                    device=device),
                   max_new_tokens=1000)[0].tolist()))


tensor([[52, 76, 64, 25,  0]])
Yui:

Wayos pte apkealaou
Y t!

Yal.

Yunto:
Ouk rheon ga wr t imowisrowenum a! se.

hahaoube?

TRileriet non to irercer sus. I nanher.
Ye c w
Akecadhe'cis!.

Mt-ugu:iwo G nraanote an a's tarenat andiveyhjoureld f y a. sn ho hharacantauinwhaonyt ehat sb se Ilyhor y g at Y gone:
:
"&itfnhaJhe ke d!

La e :
Ihatousomelrhls, tigrin pet. y nse dhiwiSo."H!.
, courie?

Regahbo:
oueaokesus!
Yug oumhit ce saled..
 Wus e ba heot I Ttao'lolmact pliukoopaere'o:

}icerarso Moce.
W tat Ile, mi:iu y.Ary cy wokares aweha b one Y, alart cyen te! capt .'cwomeht, , ec, t ibaro g gatehormt.
Woum,e is ame ot sewt lat chavo!

Tedont ae tek ikt og oso no phance was nenilhu, lons Me Cthocou ve trlseto..


u t Mano:
'het tew tr.
Me uan . wurius,u chare lhoMoo gha rint t mita p.. cn"
wa mue Olpced.


A
aousua:

Lenet Bishenv bri sle, cle Nin fhante gioumoul ls rasioin hounrtha an:
Bicg t ,is lhat Mitsa?ulyos. cnkhr Sas i?

rnitn tatouhe cake rat,'he o tanhaniq y altyt wanwu,. mad

# Fading block with multiple blocks of transfomer?

In [ ]:
class FadingLayeredBlock(nn.Module):
    def __init__(self, block_size, n_heads, n_embd, n_time, layer_num):
        super().__init__()
        self.blocks = nn.Sequential(
            *[Block(block_size, n_heads, n_embd) for _ in range(layer_num)])
        self.fade = Fade(n_time)

    def forward(self, x):
        x = self.blocks(x)
        x = self.fade(x)
        return x


In [ ]:
class FadeFormerLayeredBlocksModel(nn.Module):

    def __init__(self, vocab_size, block_size, embed_size, head_num, layer_num):
        super().__init__()
        self.block_size = block_size
        # embed raw tokens to a lower dimensional embedding with embed_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        # embed block sized context length as positional embeddings of the same size
        self.position_embedding_table = nn.Embedding(
            self.block_size, embed_size)
        # Language Modelling (?) Head is a standard linear layer to go from
        # embeddings back to logits of vocab_size
        self.lm_head = nn.Linear(embed_size, vocab_size)
        # calculate fade n_time
        fade_ins = calc_fade(self.block_size)
        # transformer blocks
        self.blocks = nn.Sequential(
            *[FadingLayeredBlock(block_size, head_num, embed_size, fade_in, layer_num) for fade_in in fade_ins])

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx)  # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(
            T, device=device))  # (T,C) [0...T-1]
        x = tok_embd + pos_embd  # (B,T,C) + (T,C) -> (B,T,C)
        # go through blocks
        x = self.blocks(x)
        # get logits with linear layer
        logits = self.lm_head(x)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets[:, -8:]
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_context = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_context)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = FadeFormerLayeredBlocksModel(vocab_size, 128, 64, 8, 2)
m = model.to(device)
idx = encode("Azusa:\n")
padded_idx = pad_encoded(idx, 128, vocab_size)
print(decode(padded_idx))
print(
    decode(
        m.generate(idx=torch.tensor([padded_idx],
                                    dtype=torch.long,
                                    device=device),
                   max_new_tokens=500)[0].tolist()))
print("model size:", sum(p.numel() for p in m.parameters()))


Azusa:



RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# training!
model = FadeFormerLayeredBlocksModel(vocab_size, 16, 16, 4, 2)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', 16)

    # evaluate the loss
    logits, loss = m(xb, yb)

    # backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (steps % 500 == 0):
        print("learning step:", steps)

print(loss.item())


learning step: 0
learning step: 500
learning step: 1000
learning step: 1500
learning step: 2000
learning step: 2500
learning step: 3000
learning step: 3500
learning step: 4000
learning step: 4500
2.8275563716888428


In [ ]:
print(loss.item())
print("model size:", sum(p.numel() for p in m.parameters()))


2.548764705657959
model size: 23721


In [ ]:
idx = encode("Yui:\n")
print(torch.tensor([idx]))
padded_idx = pad_encoded(idx, 16, vocab_size)
print(
    decode(
        m.generate(idx=torch.tensor([padded_idx],
                                    dtype=torch.long,
                                    device=device),
                   max_new_tokens=1000)[0].tolist()))


tensor([[52, 76, 64, 25,  0]])
Yui:

Mite :
Wilioiraye wes!.


Uiwa:
Houhad!

Ritssimsi:
Lw Lagkan his!
 Oaat?. gotl yosiind-se n nlroje,aoye.

Amuwoketho:
Ji:
Liky dod nna, I imi gpata!

Ri:
Whos  


Yui:
I'sim! pasatpg how le iiyekes.

Miog &a:
Baweamt rolhaite!?g? mos Lav,og..

Amurcyi:
Tu peich.

Ringmi:
Houi ma fe sas oitl ra, gatay doot!

Miiwom MslGtkeyase I the, n o ote.

Rits :
Hoow yeayos,eg yout, tewepr yeair theem'l Rlow:
Goit fwdheras,or.

Rit I Wucacld! shu docate rh wis os!

Tsum':
Wra'cen. Mmlnte whetad? .

Ria:
Bhghyye ,lom yik icalr Meteeg tondh htoc woc os!

Yuiwymi:
I, awe chyaw ". seeuav tilo!

Ritslin H?

Ami:
It! .?
Yhadr,yy Rot weafi,!.


Atsutu:
Oot?

Rii:
Ro Weger gag yiinn u'd tot!
Y slim?
I'o:
Udh, hokukac sotet tov yog am. Betevet sontedikte fame yeuk b wasorg'y ler?

Mkiko:
Wopikp. -Y whew dote dot Idom be,suraanerht siye sorem tuts tin giyam on tunamm yeemon fri yoby or our go tor depoyande! &hr'n. s Ishitheg Nakiste!

Yuik, Wisleer.

Yuitu:
Titiy..

Atss